In [30]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,TensorDataset
from cnn_model import Model
from sklearn.metrics import roc_auc_score
import datetime
import matplotlib.pyplot as plt

In [8]:
# 数据载入与处理
fea_col=np.load("data/fea_col.npy",allow_pickle=True)
train=pd.read_csv('data/train_set.csv')
test=pd.read_csv('data/val_set.csv')
train=(train.drop(columns='Label').values,train['Label'].values)
test=(test.drop(columns='Label').values,test['Label'].values)
train_set=TensorDataset(torch.tensor(train[0]).float(),torch.tensor(train[1]))
test_set=TensorDataset(torch.tensor(test[0]).float(),torch.tensor(test[1]))

In [9]:
# 超参数
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
batch_size=64
epoths=100
lr_rate=0.01
label_leaking_rate=0.1

In [27]:
# dataloader
trainloader=DataLoader(train_set,batch_size=batch_size,shuffle=True)
testloader =DataLoader(test_set ,batch_size=batch_size,shuffle=True)
# Model loader
M=Model(fea_col,gpu=torch.cuda.is_available())
try:
    M.load_state_dict("models/M.pth")
except:
    pass


In [29]:
# 训练的准备捏
history={'epoth':[],'loss':[],'test loss':[],'auc':[],'test auc':[]}
opt=torch.optim.Adam(M.parameters(),lr_rate)
loss_func=nn.BCELoss()

In [17]:
time=lambda :datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print('start time:{}'.format(time()))
for i in range(epoths):
    M.train()
    loss_sum=0.0
    auc_sum=0.0
    for step,(input,labels) in enumerate(trainloader,1):
        # 梯度清零
        opt.zero_grad()
        
        # 正向传播
        output=M(input)
        loss=loss_func(output,labels)
        auc=roc_auc_score(output,labels)
        # 反向传播
        loss.backward()
        opt.step()
    
        # 统计
        loss_sum+=loss.item()
        auc_sum+=auc.item()
    history['epoth'].append(i)
    history['loss'].append(loss_sum/step)
    history['auc'].append(auc_sum/step)
    # 验证阶段
    M.eval()
    test_loss_sum = 0.0
    test_auc_sum = 0.0
    for step,(input,labels) in enumerate(testloader,1):
        # 正向传播
        output=M(input)
        loss=loss_func(output,labels)
        auc=roc_auc_score(output,labels)

        # 统计
        test_loss_sum+=loss.item()
        test_auc_sum+=auc.item()
    history['test loss'].append(test_loss_sum/step)
    history['test auc'].append(test_auc_sum/step)
    # 打印日志
    print("Time : ",time())
    print("Epoth: ",i)
    print("Loss : ",history['loss'][i])
    print("AUC  : ",history['auc'][i])
    print("Test Loss : ",history['test loss'][i])
    print("Test AUC  : ",history['test auc'][i])
    print("")
    # 储存模型
    torch.save(M.state_dict(),"models/M.pth")


2022-10-20 16:44:44


In [ ]:
def plot_(attri):
    train=history[attri]
    test= history['test '+attri]
    epochs=range(1, len(train) + 1)
    plt.plot(epochs,train, 'bo--')
    plt.plot(epochs,test , 'ro-')
    plt.title('Training and validation '+attri)
    plt.xlabel("Epochs")
    plt.ylabel(attri)
    plt.legend(["train "+attri, 'test '+attri])
    plt.show()
plot_('loss')
plot_('auc')